In [4]:
import langchain
import langchain_community

In [6]:
## data ingestion part -
#ingesting the docs from a pdf 

In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
loader=PyPDFLoader("data/task2.pdf")
docs=loader.load()

Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)


In [3]:
docs

[Document(metadata={'producer': 'macOS Version 15.4 (Build 24E248) Quartz PDFContext', 'creator': 'Pages', 'creationdate': "D:20250417073851Z00'00'", 'title': 'task2', 'moddate': "D:20250417073851Z00'00'", 'source': 'data/task2.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='How AI is Changing the Face of \nCustomer Service – A Case Study on \nAmazon & Zappos \nAbstract\nWith the rise of digital technology, companies are under pressure to offer faster and more efﬁcient \ncustomer support. Artiﬁcial Intelligence (AI) has stepped in to ﬁll this gap. In this case study, we \nexplore how Amazon and Zappos are using AI tools like chatbots and virtual assistants to improve \ncustomer experience. We’ll look at how machine learning models help resolve customer queries \nquicker, what challenges exist in making AI feel more human, and what the future holds for AI in \ncustomer support. This study also includes a mini-project proposal where students can build a basic \nchatb

In [5]:
docs[0].page_content

'How AI is Changing the Face of \nCustomer Service – A Case Study on \nAmazon & Zappos \nAbstract\nWith the rise of digital technology, companies are under pressure to offer faster and more efﬁcient \ncustomer support. Artiﬁcial Intelligence (AI) has stepped in to ﬁll this gap. In this case study, we \nexplore how Amazon and Zappos are using AI tools like chatbots and virtual assistants to improve \ncustomer experience. We’ll look at how machine learning models help resolve customer queries \nquicker, what challenges exist in making AI feel more human, and what the future holds for AI in \ncustomer support. This study also includes a mini-project proposal where students can build a basic \nchatbot.\n1. Introduction\nCustomer service is a key factor in customer satisfaction and loyalty. With millions of daily users, \ncompanies like Amazon and Zappos need solutions that scale and respond quickly. AI is helping \nthem manage huge volumes of customer queries efﬁciently. While these tools 

In [7]:
# as the data ingestion part is completed need to do the data transofrmation part
#2. data transformation part 

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=10)

In [18]:
splitted_docs=splitter.split_documents(docs)

In [19]:
len(splitted_docs)

41

In [20]:
splitted_docs[0].page_content

'How AI is Changing the Face of \nCustomer Service – A Case Study on \nAmazon & Zappos \nAbstract\nWith the rise of digital technology, companies are under pressure to offer faster and more efﬁcient'

In [21]:
splitted_docs[1].page_content

'customer support. Artiﬁcial Intelligence (AI) has stepped in to ﬁll this gap. In this case study, we \nexplore how Amazon and Zappos are using AI tools like chatbots and virtual assistants to improve'

In [22]:
## now as the splitting of the text has been also done now time for coverting into the vecotrs

In [23]:
# 3. Embedding

In [25]:
from langchain.embeddings.ollama import OllamaEmbeddings
embedding=OllamaEmbeddings(model='llama3.2')

/var/folders/sn/kf4x3ygj0tg81pp8_fswhmp00000gn/T/ipykernel_82448/3537812255.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model='llama3.2')


In [27]:
embedded_docs=embedding.embed_documents(docs)

In [28]:
len(embedded_docs)

6

In [30]:
len(embedded_docs[0])

3072

In [31]:
## now the we know the process of the embedding the splitted docs now we can do the same with the vectorstore

In [35]:
#4. vectorstore  with embedding docs
! pip install faiss-cpu

  Using cached faiss_cpu-1.10.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.4 kB)
Using cached faiss_cpu-1.10.0-cp312-cp312-macosx_11_0_arm64.whl (3.2 MB)

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [36]:
from langchain.vectorstores import FAISS

In [38]:
db=FAISS.from_documents(splitted_docs,embedding)

In [39]:
db

In [42]:
query="What It Should Do"

In [47]:
db.similarity_search(query)[0].page_content

'browsing and purchase history.\n3. How Zappos Uses AI Differently'

In [44]:
## vectorstore as the reteriver
retriever=db.as_retriever()

In [46]:
retriever.invoke(query)[0].page_content

'browsing and purchase history.\n3. How Zappos Uses AI Differently'

In [52]:
## using the another vectorstore which is chormadb
! pip install langchain_chroma
! pip install chromadb


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [54]:
from langchain_chroma import Chroma

In [56]:
chromadb=Chroma.from_documents(splitted_docs,embedding)

In [59]:
chromadb.similarity_search(query)

[Document(id='b608ed92-cde0-463e-add4-b60499c4a5f2', metadata={'creationdate': "D:20250417073851Z00'00'", 'creator': 'Pages', 'moddate': "D:20250417073851Z00'00'", 'page': 1, 'page_label': '2', 'producer': 'macOS Version 15.4 (Build 24E248) Quartz PDFContext', 'source': 'data/task2.pdf', 'title': 'task2', 'total_pages': 6}, page_content='browsing and purchase history.\n3. How Zappos Uses AI Differently'),
 Document(id='030f34cb-3765-4dec-8c10-85e9cb13911c', metadata={'creationdate': "D:20250417073851Z00'00'", 'creator': 'Pages', 'moddate': "D:20250417073851Z00'00'", 'page': 3, 'page_label': '4', 'producer': 'macOS Version 15.4 (Build 24E248) Quartz PDFContext', 'source': 'data/task2.pdf', 'title': 'task2', 'total_pages': 6}, page_content='This helps companies decide when to get a human involved or when to offer a discount or apology.\n6.2 Quick Replies with Auto-Response Tools'),
 Document(id='d6ac9d12-571b-43ab-a6e6-a2b4a6dffe72', metadata={'creationdate': "D:20250417073851Z00'00'", '

In [60]:
retriver=chromadb.as_retriever()

In [61]:
retriever.invoke(query)[0].page_content

'browsing and purchase history.\n3. How Zappos Uses AI Differently'